<a href="https://colab.research.google.com/github/Dmitry1937/NLP-se-to-se-/blob/main/Sec_to_Sec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  ## Подгтовка DF


In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/My\ Drive/Skillbox/ChatBot_Dataset.zip -d ChatBot_Dataset

Archive:  /content/drive/My Drive/Skillbox/ChatBot_Dataset.zip
replace ChatBot_Dataset/SOURCE.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#raw_script_urls = pd.read_csv('ChatBot_Dataset/raw_script_urls.txt', sep = ' \+\+\+\$\+\+\+ ',header=None, engine='python'  ) 
#raw_script_urls.head()
#movie_titles_metadata = pd.read_csv('ChatBot_Dataset/movie_titles_metadata.txt', sep = ' \+\+\+\$\+\+\+ ',header=None, engine='python'  ) 
#movie_titles_metadata.head()
#movie_characters_metadata = pd.read_csv('ChatBot_Dataset/movie_characters_metadata.txt', sep = ' \+\+\+\$\+\+\+ ',header=None, engine='python'  ) 
#movie_characters_metadata.head()

In [ ]:
#read movie_conversations.txt
movie_conversations = pd.read_csv('ChatBot_Dataset/movie_conversations.txt', sep = ' \+\+\+\$\+\+\+ ',header=None, engine='python' ,parse_dates=True ) 
movie_conversations.head()

,0,1,2,3
0,u0,u2,m0,"['L194', 'L195', 'L196', 'L197']"
1,u0,u2,m0,"['L198', 'L199']"
2,u0,u2,m0,"['L200', 'L201', 'L202', 'L203']"
3,u0,u2,m0,"['L204', 'L205', 'L206']"
4,u0,u2,m0,"['L207', 'L208']"


In [ ]:
def parser (srtings):
  #Функция превращает текст в список с индексами строчек
  new = []
  number =''
  for c in srtings[2:-2]:
    if not (c in '\', '):
      number +=c
    elif number != '' :
      new.append(number)
      number = ''
  new.append(number)
  return new


def set_pair (dialogs:str) : 
    #Функция создает все возможные пары для диалогов с одной строчки
    set_pairs = list()
    dialogs = parser (dialogs)
    n_old = dialogs[0]
    for n in dialogs[1:]:
      set_pairs.append([n_old,n])
      n_old = n
      
    return set_pairs


def all_pair (col):
  #Функция создает список всех возможных пар в предложенном сете
  all_pairs = []
  for c in col:
    for n in c:

      all_pairs.append(n)
  
  return all_pairs


In [ ]:
#С помощью функций созданных выше формируем DataFrame со всеми парами
all_pairs = movie_conversations.agg({3 : set_pair })
all_pairs = pd.DataFrame(all_pairs.aggregate(all_pair, axis =0, ).values.squeeze().tolist() , columns=['s','f'])
all_pairs

,s,f
0,L194,L195
1,L195,L196
2,L196,L197
3,L198,L199
4,L200,L201
...,...,...
221611,L666369,L666370
221612,L666370,L666371
221613,L666371,L666372
221614,L666520,L666521


In [ ]:
#read movie_lines.txt
colums = ['0','1','2','3','4',]
movie_lines = pd.read_csv('ChatBot_Dataset/movie_lines.txt', sep = ' \+\+\+\$\+\+\+ ',header=None, engine='python' , ) 
movie_lines.columns = colums
movie_lines.dropna(inplace=True)
movie_lines.head()

,0,1,2,3,4
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [ ]:
#Создаем DataFrame для работы
df = all_pairs.merge(movie_lines[['0','4']],left_on='s', right_on='0',  )
df = df.merge(movie_lines[['0','4']],left_on='f', right_on='0',)
df.drop(['s', 'f','0_x','0_y'], axis=1,inplace=True)
df.columns = ['start','finish']
df.head()

,start,finish
0,Can we make this quick? Roxanne Korrine and A...,"Well, I thought we'd start with pronunciation,..."
1,"Well, I thought we'd start with pronunciation,...",Not the hacking and gagging and spitting part....
2,Not the hacking and gagging and spitting part....,Okay... then how 'bout we try out some French ...
3,You're asking me out. That's so cute. What's ...,Forget it.
4,"No, no, it's my fault -- we didn't have a prop...",Cameron.


### Подготовим текс к ML

In [ ]:

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
import tensorflow as tf
'''import attention 
import nltk
import string
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.utils import class_weight'''
from keras import initializers, regularizers, constraints


#nltk.download('punkt')

In [ ]:
def plus_words (text):
  return '11star11 '+text+' 11finish11'
df = df.apply(plus_words)

In [ ]:
# prepare tokenizer
t = Tokenizer( )
t.fit_on_texts(df.start	 +' '+ df.finish)
vocab_size = len(t.word_index) + 1
vocab_size

55748

In [ ]:
# integer encode the documents
encoded_docs_start = t.texts_to_sequences(df.start, )
encoded_docs_finish = t.texts_to_sequences(df.finish, )

In [ ]:
max_length = 0
for i in encoded_docs_start:
  max_length +=len(i)
max_length = int(max_length/len(encoded_docs_start))
max_length

12

In [ ]:
MAX_LEN = 7 # Для чат бота 5 слов + старт стоп
MAX_LEN 

7

In [ ]:
too_long = []
for indx,i in enumerate(encoded_docs_start):
  if len(i)>MAX_LEN:
    too_long.append(indx)
len(too_long)

for indx,i in enumerate(encoded_docs_finish):
  if len(i)>MAX_LEN:
    too_long.append(indx)

In [ ]:
# Что бы не путать сеть с незаконченными фразами принято решения удолить пары где более 20 слов на start или finish
too_long = set(too_long)
df = df.drop(index=too_long,axis=0 ,).reset_index()
df

,index,start,finish
0,15,11star11 There. 11finish11,11star11 Where? 11finish11
1,26,11star11 Have fun tonight? 11finish11,11star11 Tons 11finish11
2,31,11star11 But 11finish11,11star11 You always been this selfish? 11finish11
3,37,11star11 What good stuff? 11finish11,"11star11 The ""real you"". 11finish11"
4,41,11star11 Wow 11finish11,11star11 Let's go. 11finish11
...,...,...,...
39143,221181,"11star11 Yes, sir, name, please? 11finish11",11star11 Food! 11finish11
39144,221182,11star11 Food! 11finish11,11star11 Do you have a reservation? 11finish11
39145,221183,11star11 Do you have a reservation? 11finish11,11star11 Food!! 11finish11
39146,221188,11star11 GRRRHMMNNNJKJMMMNN! 11finish11,11star11 Franz! Help! Lunatic! 11finish11


In [ ]:
#Повторим операцию с подготовленным df
t = Tokenizer( )
t.fit_on_texts(df.start	 +' '+ df.finish)
vocab_size = len(t.word_index) + 1
vocab_size

13610

In [ ]:
# integer encode the documents
encoded_docs_start = t.texts_to_sequences(df.start, )
encoded_docs_finish = t.texts_to_sequences(df.finish, )

padded_docs_start = pad_sequences(encoded_docs_start, maxlen=MAX_LEN, padding='post', )
padded_docs_finish  = pad_sequences(encoded_docs_finish, maxlen=MAX_LEN+1, padding='post', )

encoded_docs_finish_input = np.array([i[:-1] for i in padded_docs_finish])
encoded_docs_finish_target = np.array([i[1:] for i in padded_docs_finish])

In [ ]:
#Проверка
max_length = 0
for i in encoded_docs_start:
  if max_length< len(i):
    max_length =  len(i)
print(max_length)


max_length = 0
for i in encoded_docs_finish:
  if max_length< len(i):
    max_length =  len(i)
print(max_length)

7
7


In [ ]:
'''# pad documents to a max length of words
padded_docs_start = pad_sequences(encoded_docs_start, maxlen=MAX_LEN, padding='post', )
padded_docs_finish_input  = pad_sequences(encoded_docs_finish_input, maxlen=MAX_LEN, padding='post', )
padded_docs_finish_target  = pad_sequences(encoded_docs_finish_target, maxlen=MAX_LEN, padding='post', )'''

"# pad documents to a max length of words\npadded_docs_start = pad_sequences(encoded_docs_start, maxlen=MAX_LEN, padding='post', )\npadded_docs_finish_input  = pad_sequences(encoded_docs_finish_input, maxlen=MAX_LEN, padding='post', )\npadded_docs_finish_target  = pad_sequences(encoded_docs_finish_target, maxlen=MAX_LEN, padding='post', )"

In [ ]:
MAX_SEQ_LEN = MAX_LEN #1000
EMB_SIZE = 50 # Размер векторного представления (эмбеддинга)
BATCH_SIZE = 256

### GloVE как  Embedding

In [ ]:
import os, requests, shutil

glove_dir = './data/RNN/'
glove_100k_50d = 'glove.first-100k.6B.50d.txt'
glove_100k_50d_path = os.path.join(glove_dir, glove_100k_50d)

# These are temporary files if we need to download it from the original source (slow)
#The original file is at http://nlp.stanford.edu/data/glove.840B.300d.zip
data_cache = './data/cache'
glove_full_tar = 'glove.6B.zip'
glove_full_50d = 'glove.6B.50d.txt'

#force_download_from_original=False
download_url= 'http://redcatlabs.com/downloads/deep-learning-workshop/notebooks/data/RNN/'+glove_100k_50d
original_url = 'http://nlp.stanford.edu/data/'+glove_full_tar

if not os.path.isfile( glove_100k_50d_path ):
    if not os.path.exists(glove_dir):
        os.makedirs(glove_dir)
    
    # First, try to download a pre-prepared file directly...
    response = requests.get(download_url, stream=True)
    if response.status_code == requests.codes.ok:
        print("Downloading 42Mb pre-prepared GloVE file from RedCatLabs")
        with open(glove_100k_50d_path, 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
    else:
        # But, for some reason, RedCatLabs didn't give us the file directly
        if not os.path.exists(data_cache):
            os.makedirs(data_cache)
        
        if not os.path.isfile( os.path.join(data_cache, glove_full_50d) ):
            zipfilepath = os.path.join(data_cache, glove_full_tar)
            if not os.path.isfile( zipfilepath ):
                print("Downloading 860Mb GloVE file from Stanford")
                response = requests.get(download_url, stream=True)
                with open(zipfilepath, 'wb') as out_file:
                    shutil.copyfileobj(response.raw, out_file)
            if os.path.isfile(zipfilepath):
                print("Unpacking 50d GloVE file from zip")
                import zipfile
                zipfile.ZipFile(zipfilepath, 'r').extract(glove_full_50d, data_cache)

        with open(os.path.join(data_cache, glove_full_50d), 'rt') as in_file:
            with open(glove_100k_50d_path, 'wt') as out_file:
                print("Reducing 50d GloVE file to first 100k words")
                for i, l in enumerate(in_file.readlines()):
                    if i>=100000: break
                    out_file.write(l)
    
        # Get rid of tarfile source (the required text file itself will remain)
        #os.unlink(zipfilepath)
        #os.unlink(os.path.join(data_cache, glove_full_50d))

print("GloVE available locally")

GloVE available locally


In [ ]:
filename = 'data/RNN/glove.first-100k.6B.50d.txt'

In [ ]:
#encoded_docs_finish_input = [i[:-1] for i in padded_docs_finish]
#encoded_docs_finish_target = [i[1:] for i in padded_docs_finish]
len(encoded_docs_finish_target)

39148

In [ ]:
# split the data into a training set and a validation set
VALIDATION_SPLIT = 0.2
indices = np.arange(padded_docs_start.shape[0])
np.random.shuffle(indices)
padded_docs_start = padded_docs_start[indices]
encoded_docs_finish_input = encoded_docs_finish_input[indices]
encoded_docs_finish_target = encoded_docs_finish_target[indices]

nb_validation_samples = int(VALIDATION_SPLIT * padded_docs_start.shape[0])

x_train = padded_docs_start[:-nb_validation_samples]
y_train_input = encoded_docs_finish_input[:-nb_validation_samples]
y_train_target = encoded_docs_finish_target[:-nb_validation_samples]
x_val = padded_docs_start[-nb_validation_samples:]
y_val_input = encoded_docs_finish_input[-nb_validation_samples:]
y_val_target = encoded_docs_finish_target[-nb_validation_samples:]

In [ ]:
# load the whole embedding into memory
embeddings_index = dict()
f = open(filename)
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.array(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 100000 word vectors.


In [ ]:
# create a weight matrix for words in training docs
EMB_SIZE = 50 # размерность эмбеддингов (и для входных и для выходных цепочек)
embedding_matrix = np.zeros((vocab_size, EMB_SIZE))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
embedding_matrix[t.word_index['11star11']] = np.zeros(50)
embedding_matrix[t.word_index['11finish11']] = np.ones(50)

In [ ]:
from keras.layers import Activation, dot, concatenate

## Train

In [ ]:
H_SIZE = 512 # Размерность скрытого состояния LSTM

class Encoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(vocab_size, EMB_SIZE,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False
                                               )
        self.lstm1 = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True, dropout=0.05)
        self.lstm2 = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True, dropout=0.05)

        
    def call(self, x):
        out = self.embed(x)
        out, h, c, = self.lstm1(out)
        state  = self.lstm2(out)
        
        return state

class Decoder(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.embed = tf.keras.layers.Embedding(vocab_size, EMB_SIZE,weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False
                                               )
        self.lstm = tf.keras.layers.LSTM(H_SIZE, return_sequences=True, return_state=True,dropout=0.05)
        self.attention = tf.keras.layers.Dense(max_length, activation='softmax')
        self.fc = tf.keras.layers.Dense(vocab_size, activation='softmax')
        
    def call(self, x, init_state):
        out = self.embed(x)
        
        out, h, c = self.lstm(out, initial_state=(init_state[1:]),)

        atten = dot([out, init_state[0]], axes=[2, 2])
        attention = Activation('softmax', name='attention')(atten)
        context = dot([attention, init_state[0]], axes=[2,1])

        decoder_combined_context = concatenate([context, out])

        out = self.fc(decoder_combined_context)
        state = (h, c)
        return out, state

encoder_model = Encoder()
decoder_model = Decoder()

encoder_inputs = tf.keras.layers.Input(shape=(None,))
decoder_inputs = tf.keras.layers.Input(shape=(None,))

enc_state = encoder_model(encoder_inputs)
decoder_outputs, _ = decoder_model(decoder_inputs, enc_state)

seq2seq = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
BATCH_SIZE = 256
EPOCHS = 45

loss = tf.losses.SparseCategoricalCrossentropy()
seq2seq.compile(optimizer='nadam', loss=loss, metrics=['accuracy'])
seq2seq.fit([x_train, y_train_input], y_train_target, 
          batch_size=BATCH_SIZE, 
          epochs=EPOCHS)

Epoch 1/45
123/123 [==============================] - 14s 73ms/step - loss: 4.1029 - accuracy: 0.5037
Epoch 2/45
123/123 [==============================] - 9s 73ms/step - loss: 2.8930 - accuracy: 0.5861
Epoch 3/45
123/123 [==============================] - 9s 73ms/step - loss: 2.7985 - accuracy: 0.5897
Epoch 4/45
123/123 [==============================] - 9s 73ms/step - loss: 2.7030 - accuracy: 0.5937
Epoch 5/45
123/123 [==============================] - 9s 73ms/step - loss: 2.6361 - accuracy: 0.5925
Epoch 6/45
123/123 [==============================] - 9s 72ms/step - loss: 2.5489 - accuracy: 0.5958
Epoch 7/45
123/123 [==============================] - 9s 72ms/step - loss: 2.4687 - accuracy: 0.5973
Epoch 8/45
123/123 [==============================] - 9s 72ms/step - loss: 2.3845 - accuracy: 0.6000
Epoch 9/45
123/123 [==============================] - 9s 72ms/step - loss: 2.3044 - accuracy: 0.6021
Epoch 10/45
123/123 [==============================] - 9s 72ms/step - loss: 2.2172 - accur

In [ ]:
def seq2seq_inference(input_seq):
    state = encoder_model(input_seq)
    target_seq = np.array([[1]])
    out = state[0]
    decoded_sentence = ''

    while True:
        output_tokens,state = decoder_model(target_seq, state)
        
        state = out,*state
        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        if sampled_token_index>0 :
          sampled_char = t.index_word[sampled_token_index]
        else :
          sampled_char = 'post'

        #decoded_sentence += sampled_char +' '
        if (sampled_char == '11finish11' or
           len(decoded_sentence) > MAX_LEN):
            #print(decoded_sentence)
            break
        decoded_sentence += sampled_char +' '

        target_seq = np.array([[sampled_token_index]])

    return decoded_sentence

In [ ]:
for seq_index in indices[-15:-20:-1]:
    input_seq = padded_docs_start[seq_index: seq_index + 1]
    decoded_sentence = seq2seq_inference(input_seq)
    print('-')
    print('Input sentence:', df.start[indices].iloc[seq_index].split('11finish11')[0][8:])
    print('Result sentence:', decoded_sentence)
    print('Target sentence:', df.finish[indices].iloc[seq_index].split('11finish11')[0][8:])
    print('-')

-
Input sentence:  Yes. 
Result sentence: you personally 
Target sentence:  Why? 
-
-
Input sentence:  About tonsils. 
Result sentence: u tonsili 
Target sentence:  <u>Tonsili!</u> 
-
-
Input sentence:  Very well, m'sieu. 
Result sentence: otherwise 
Target sentence:  Otherwise I'll call the police 
-
-
Input sentence:  David? 
Result sentence: ellie 
Target sentence:  Who's there? 
-
-
Input sentence:  You're shaking. 
Result sentence: i don't 
Target sentence:  It's the tension. 
-


In [ ]:
for seq_index in indices[-25:-40:-1]:
    input_seq = padded_docs_start[seq_index: seq_index + 1]
    decoded_sentence = seq2seq_inference(input_seq)
    print('-')
    print('Input sentence:', df.start[indices].iloc[seq_index].split('11finish11')[0][8:])
    print('Result sentence:', decoded_sentence)
    print('Target sentence:', df.finish[indices].iloc[seq_index].split('11finish11')[0][8:])
    print('-')

-
Input sentence:  Syevodnya? 
Result sentence: what 
Target sentence:  Syevodnya 
-
-
Input sentence:  That's me. 
Result sentence: i'm glad 
Target sentence:  No it is not! 
-
-
Input sentence:  Can't you see? 
Result sentence: father of 
Target sentence:  Oh, yeah. There. 
-
-
Input sentence:  A hotel? 
Result sentence: it's a loft 
Target sentence:  It's not used for anything. 
-
-
Input sentence:  Murphy to Epps. 
Result sentence: epps 
Target sentence:  Epps. 
-
-
Input sentence:  What about Miss Lawson? 
Result sentence: yes cocaine 
Target sentence:  Yes -- cocaine. 
-
-
Input sentence:  Not these guys. 
Result sentence: yeah those 
Target sentence:  Yeah, those guys too. 
-
-
Input sentence:  Yes. 
Result sentence: you personally 
Target sentence:  No further questions. 
-
-
Input sentence:  It's the game, ARCADE. 
Result sentence: you don't 
Target sentence:  You don't like it? 
-
-
Input sentence:  Who's next, Mrs. Rabinow. 
Result sentence: her 
Target sentence:  We rest, Y

## Summary

Проверять на таких данных не совсем корректно, так как есть leak (связанная с тем что данные  начало диалога которые val могут быть в train df как конец диалога и наоборот. Но тем не менее несмотря на отсутствия контекста иногда получается достаточно осмысленной ответ. Я изначально упростил задачу исключив длинные предложения, так как это бы значительно увеличило вектор ответов. В чем нет необходимости для Chat bota